In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
import datetime
import pprint
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# Skymaps

In [3]:
# search datasets for skymaps
aurorax.data.list_datasets_in_table(name="SKYMAP")

Name                       Provider   Level   DOI Details                                               Short Description                                          
REGO_SKYMAP_IDLSAV         UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/z7x6-5c42   REGO All Sky Imagers skymap data (IDL save format)         
THEMIS_ASI_SKYMAP_IDLSAV   UCalgary   L3      None                                                      THEMIS All Sky Imagers skymap data (IDL save format)       
TREX_BLUE_SKYMAP_IDLSAV    UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/80pf-0p02   TREx Blueline All Sky Imagers skymap data (IDL save format)
TREX_NIR_SKYMAP_IDLSAV     UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/98w7-jp47   TREx NIR All Sky Imagers skymap data (IDL save format)     
TREX_RGB_SKYMAP_IDLSAV     UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/4p8e-1k65   TREx RGB All Sky Imagers skymap data (IDL save format)     


In [4]:
# we'll set our dataset for use later
dataset = aurorax.data.list_datasets(name="THEMIS_ASI_SKYMAP_IDLSAV")[0]

When selecting a skymap to use for projecting an image on a map, we have two methods available to us:

1) choosing manually
2) using the `find_best_skymap()` function to choose automatically

Skymaps are generated for each site, and for a given time period. It is important to choose a skymap that is valid for the date you're looking at data for, otherwise the image may not appear accurately when projected on a map.


## Choosing a skymap manually

In [5]:
# First, let's choose the skymap we want manually. Let's assume we are working on data
# from the Gillam THEMIS ASI on 2021-11-04.
#
# We'll download the skymaps for a few years around that time, and then we'll choose which one we want after
r = aurorax.data.ucalgary.download(
    "THEMIS_ASI_SKYMAP_IDLSAV",
    datetime.datetime(2021, 1, 1, 0, 0),
    datetime.datetime(2023, 1, 1, 0, 0),
    site_uid="gill",
)
r.filenames

[WindowsPath('C:/Users/darrenc/pyaurorax_data/THEMIS_ASI_SKYMAP_IDLSAV/gill/gill_20210308/themis_skymap_gill_20210308-+_v02.sav'),
 WindowsPath('C:/Users/darrenc/pyaurorax_data/THEMIS_ASI_SKYMAP_IDLSAV/gill/gill_20220301/themis_skymap_gill_20220301-+_v02.sav')]

Looks like we have a couple skymaps to choose from around 2021-11-04. We'll choose the first one since the date for it is before, and the second one's date is after. 

The date indicates the first date it is valid for. There are some cases where a later or earlier skymap can be used. That is a situation where you can play around and try different skymaps, looking for which one works best for you. Most skymaps have small differences, but some have large ones that you'll notice very quickly when working with the projected data on a map.


In [6]:
# Now that we know which one we'll use, we can read it in.
#
# You can also read in all of them and choose later using the resulting Data object.
skymap_data = aurorax.data.ucalgary.read(dataset, r.filenames[0])

print(skymap_data)
print()
skymap_data.pretty_print()

print()
skymap_data.data[0].pretty_print()

Data(data=[1 Skymap object], timestamp=[], metadata=[], problematic_files=[], calibrated_data=None, dataset=Dataset(name=THEMIS_ASI_SKYMAP_IDLSAV, short_description='THEMIS All Sky Im...))

Data:
  data                  : [1 Skymap object]
  timestamp             : []
  metadata              : []
  problematic_files     : []
  calibrated_data       : None
  dataset               : Dataset(name=THEMIS_ASI_SKYMAP_IDLSAV, short_description='THEMIS All Sky Im...)

Skymap:
  filename               : C:\Users\darrenc\pyaurorax_data\THEMIS_ASI_SKYMAP_IDLSAV\gill\gill_20210308\themis_skymap_gill_20210308-+_v02.sav
  full_azimuth           : array(dims=(256, 256), dtype=>f4)
  full_elevation         : array(dims=(256, 256), dtype=>f4)
  full_map_altitude      : array(dims=(3,), dtype=>f4)
  full_map_latitude      : array(dims=(3, 257, 257), dtype=>f4)
  full_map_longitude     : array(dims=(3, 257, 257), dtype=>f4)
  generation_info        : SkymapGenerationInfo(...)
  imager_uid             : t

## Automatically choosing a skymap

You can also let the library choose the skymap for you using the `download_best_skymap()` function.

In [7]:
# set params
dataset_name = "THEMIS_ASI_SKYMAP_IDLSAV"
site_uid = "gill"
dt = datetime.datetime(2021, 11, 4)

# get the recommendation
r = aurorax.data.ucalgary.download_best_skymap(dataset_name, site_uid, dt)
r.filenames


[WindowsPath('C:/Users/darrenc/pyaurorax_data/THEMIS_ASI_SKYMAP_IDLSAV/gill/gill_20210308/themis_skymap_gill_20210308-+_v02.sav')]

In [8]:
# now that we have the skymap file, we'll read it
skymap_data = aurorax.data.ucalgary.read(dataset, r.filenames)

print(skymap_data)
print()
skymap_data.pretty_print()

print()
skymap_data.data[0].pretty_print()

Data(data=[1 Skymap object], timestamp=[], metadata=[], problematic_files=[], calibrated_data=None, dataset=Dataset(name=THEMIS_ASI_SKYMAP_IDLSAV, short_description='THEMIS All Sky Im...))

Data:
  data                  : [1 Skymap object]
  timestamp             : []
  metadata              : []
  problematic_files     : []
  calibrated_data       : None
  dataset               : Dataset(name=THEMIS_ASI_SKYMAP_IDLSAV, short_description='THEMIS All Sky Im...)

Skymap:
  filename               : C:\Users\darrenc\pyaurorax_data\THEMIS_ASI_SKYMAP_IDLSAV\gill\gill_20210308\themis_skymap_gill_20210308-+_v02.sav
  full_azimuth           : array(dims=(256, 256), dtype=>f4)
  full_elevation         : array(dims=(256, 256), dtype=>f4)
  full_map_altitude      : array(dims=(3,), dtype=>f4)
  full_map_latitude      : array(dims=(3, 257, 257), dtype=>f4)
  full_map_longitude     : array(dims=(3, 257, 257), dtype=>f4)
  generation_info        : SkymapGenerationInfo(...)
  imager_uid             : t

# Calibrations

In [9]:
# search datasets for calibrations
aurorax.data.list_datasets_in_table(name="CALIBRATION")

Name                                     Provider   Level   DOI Details                                               Short Description                                                         
REGO_CALIBRATION_FLATFIELD_IDLSAV        UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/z7x6-5c42   REGO All Sky Imagers Flatfield calibration data (IDL save format)         
REGO_CALIBRATION_RAYLEIGHS_IDLSAV        UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/z7x6-5c42   REGO All Sky Imagers Rayleighs calibration data (IDL save format)         
TREX_BLUE_CALIBRATION_FLATFIELD_IDLSAV   UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/80pf-0p02   TREx Blueline All Sky Imagers Flatfield calibration data (IDL save format)
TREX_BLUE_CALIBRATION_RAYLEIGHS_IDLSAV   UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/80pf-0p02   TREx Blueline All Sky Imagers Rayleighs calibration data (IDL save format)
TREX_NIR_CALIBRATION_FLATFIELD_IDLS

When selecting calibration files to use for converting ASI image counts to Rayleighs, we have two methods available to us:

1) choosing manually
2) using the `find_best_flatfield_calibration()` or `find_best_rayleighs_calibration()` function to choose automatically

Cameras are calibrated before they are deployed to the field, and after any in-house repairs are performed. There exist flatfield and Rayleighs calibration files, for each specific camera detector. A detector can live at multiple sites thoughout the years of operating the instrument array. Hence, why they are not associated with a specific site at all. We use the data device UID value to know what calibration files to use for the data we're processing.

In [10]:
# download a minute of REGO data
dataset_name = "REGO_RAW"
start_dt = datetime.datetime(2021, 11, 4, 6, 0)
end_dt = datetime.datetime(2021, 11, 4, 6, 0)
site_uid = "resu"
r = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid, progress_bar_disable=True)

# determine the device uid
#
# you can either inspect the URLs and determine it by the filename, or you can read the
# data and inspect the device UID field of the metadata
print(r.filenames[0])
print()

data = aurorax.data.ucalgary.read(r.dataset, r.filenames)
print(data.metadata[0]["Imager unique ID"])

C:\Users\darrenc\pyaurorax_data\REGO_RAW\2021\11\04\resu_rego-655\ut06\20211104_0600_resu_rego-655_6300.pgm.gz

rego-655


In [11]:
# now that we know the device UID we are interested in, we can get a list
# of all the flatfield and rayleighs calibration files, and then choose which
# one we want to download
start_dt = datetime.datetime(2014, 1, 1, 0, 0)
end_dt = datetime.datetime.now()
device_uid = "655"
r_rayleighs = aurorax.data.ucalgary.get_urls("REGO_CALIBRATION_RAYLEIGHS_IDLSAV", start_dt, end_dt, device_uid=device_uid)
pprint.pprint(r_rayleighs.urls)

r_flatfield = aurorax.data.ucalgary.get_urls("REGO_CALIBRATION_FLATFIELD_IDLSAV", start_dt, end_dt, device_uid=device_uid)
pprint.pprint(r_flatfield.urls)

['https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/REGO/calibration/REGO_Rayleighs_15655_20141002-+_v01.sav']
['https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/REGO/calibration/REGO_flatfield_15655_20141002-+_v01.sav']


In [12]:
# this is simple as there is only one to choose from
#
# now let's download the data
d_rayleighs = aurorax.data.ucalgary.download_using_urls(r_rayleighs, progress_bar_disable=True)
d_flatfield = aurorax.data.ucalgary.download_using_urls(r_flatfield, progress_bar_disable=True)

print(d_rayleighs.filenames)

print(d_flatfield.filenames)

[WindowsPath('C:/Users/darrenc/pyaurorax_data/REGO_CALIBRATION_RAYLEIGHS_IDLSAV/REGO_Rayleighs_15655_20141002-+_v01.sav')]
[WindowsPath('C:/Users/darrenc/pyaurorax_data/REGO_CALIBRATION_FLATFIELD_IDLSAV/REGO_flatfield_15655_20141002-+_v01.sav')]


In [13]:
# now that we have the calibration files, we'll read them
cal_rayleighs_data = aurorax.data.ucalgary.read(d_rayleighs.dataset, d_rayleighs.filenames)
cal_flatfield_data = aurorax.data.ucalgary.read(d_flatfield.dataset, d_flatfield.filenames)

print(cal_rayleighs_data)
print(cal_flatfield_data)
print()

cal_rayleighs_data.data[0].pretty_print()
print()
cal_flatfield_data.data[0].pretty_print()


Data(data=[1 Calibration object], timestamp=[], metadata=[], problematic_files=[], calibrated_data=None, dataset=Dataset(name=REGO_CALIBRATION_RAYLEIGHS_IDLSAV, short_description='REGO All...))
Data(data=[1 Calibration object], timestamp=[], metadata=[], problematic_files=[], calibrated_data=None, dataset=Dataset(name=REGO_CALIBRATION_FLATFIELD_IDLSAV, short_description='REGO All...))

Calibration:
  dataset                       : Dataset(...)
  detector_uid                  : 15655
  filename                      : C:\Users\darrenc\pyaurorax_data\REGO_CALIBRATION_RAYLEIGHS_IDLSAV\REGO_Rayleighs_15655_20141002-+_v01.sav
  flat_field_multiplier         : None
  generation_info               : CalibrationGenerationInfo(...)
  rayleighs_perdn_persecond     : 10.399999618530273
  version                       : v01

Calibration:
  dataset                       : Dataset(...)
  detector_uid                  : 15655
  filename                      : C:\Users\darrenc\pyaurorax_data\REGO_CALI

## Automatically choosing a calibration file

You can also let the library choose the calibration for you using the `download_best_flatfield_calibration()` and `download_best_rayleighs_calibration()` functions.

In [14]:
# set params
device_uid = "654"
dt = datetime.datetime(2021, 11, 4)

# get the recommendations
r_rayleighs = aurorax.data.ucalgary.download_best_rayleighs_calibration("REGO_CALIBRATION_RAYLEIGHS_IDLSAV", device_uid, dt)
r_flatfield = aurorax.data.ucalgary.download_best_flatfield_calibration("REGO_CALIBRATION_FLATFIELD_IDLSAV", device_uid, dt)

# show results
print(r_rayleighs.filenames)
print(r_flatfield.filenames)

[WindowsPath('C:/Users/darrenc/pyaurorax_data/REGO_CALIBRATION_RAYLEIGHS_IDLSAV/REGO_Rayleighs_15654_20210806-+_v02.sav')]
[WindowsPath('C:/Users/darrenc/pyaurorax_data/REGO_CALIBRATION_FLATFIELD_IDLSAV/REGO_flatfield_15654_20210806-+_v02.sav')]


In [15]:
# now that we have the calibration file, we'll read it
cal_rayleighs_data = aurorax.data.ucalgary.read(r_rayleighs.dataset, r_rayleighs.filenames)
cal_flatfield_data = aurorax.data.ucalgary.read(r_flatfield.dataset, r_flatfield.filenames)

cal_rayleighs_data.data[0].pretty_print()
print()
cal_flatfield_data.data[0].pretty_print()

Calibration:
  dataset                       : Dataset(...)
  detector_uid                  : 15654
  filename                      : C:\Users\darrenc\pyaurorax_data\REGO_CALIBRATION_RAYLEIGHS_IDLSAV\REGO_Rayleighs_15654_20210806-+_v02.sav
  flat_field_multiplier         : None
  generation_info               : CalibrationGenerationInfo(...)
  rayleighs_perdn_persecond     : 10.137431837782506
  version                       : v02

Calibration:
  dataset                       : Dataset(...)
  detector_uid                  : 15654
  filename                      : C:\Users\darrenc\pyaurorax_data\REGO_CALIBRATION_FLATFIELD_IDLSAV\REGO_flatfield_15654_20210806-+_v02.sav
  flat_field_multiplier         : array(dims=(512, 512), dtype=>f8)
  generation_info               : CalibrationGenerationInfo(...)
  rayleighs_perdn_persecond     : None
  version                       : v02
